In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("minmax_wine_df.csv",index_col=[0])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23394 entries, 0 to 23781
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         23394 non-null  object 
 1   price        23394 non-null  float64
 2   score        23394 non-null  float64
 3   winery       23394 non-null  object 
 4   grapes       23394 non-null  object 
 5   country      23394 non-null  object 
 6   region       23394 non-null  object 
 7   alcohol      23394 non-null  float64
 8   foods        23394 non-null  object 
 9   wine_id      23394 non-null  int64  
 10  re           23394 non-null  object 
 11  rev_alcohol  23394 non-null  float64
dtypes: float64(4), int64(1), object(7)
memory usage: 2.3+ MB


In [5]:
df.head()

,name,price,score,winery,grapes,country,region,alcohol,foods,wine_id,re,rev_alcohol
0,Rim Rock Vineyard Syrah 2017,52.99,0.33,Piedrasassi,Shiraz/Syrah,United States,Arroyo Grande Valley,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",19603399,['Dry tingly cool bottle pairs with seafood ho...,0.70
1,Tradition Châteauneuf-du-Pape 2015,54.99,0.22,Domaine Giraud,Grenache,France,Châteauneuf-du-Pape,0.30,"['Lamb', 'Pork', 'Poultry']",18191466,"['Black cherry, oak. Leather, earthy. Great b...",0.70
2,Estate Cabernet Sauvignon 2013,73.95,0.33,Brandlin,Cabernet Sauvignon,United States,Mount Veeder,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Matu...",13886692,['Wine walk favorite! This estate cab is fant...,0.70
3,Imperio LXXIV Primitivo di Manduria 2018,19.99,0.22,Feudo Croce,Primitivo,Italy,Primitivo di Manduria,0.32,"['Beef', 'Pasta', 'Lamb', 'Poultry']",25311528,"['Plum, fig, pepper, cherry. ', 'Lots of dark ...",0.68
4,Corte di Cama Sforzato di Valtellina 2016,58.50,0.33,Mamete Prevostini,Nebbiolo,Italy,Sforzato della Valtellina,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",25588649,['A bit of a disappointment honestly. Have rea...,0.70


In [6]:
import re
import nltk

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

여기서 사용자에게 와인이름이 한개 나오면 이거 맞냐고 물어보고
와인이름이 여러개 나오면 이 중 맞는 걸 선택해달라고 하기!

In [8]:
#불용어 제거 

nltk.download('stopwords')

#from nltk.corpus import stopwords

stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
wine_name = input("와인 이름을 입력해주세요: ")

와인 이름을 입력해주세요: Rim


In [10]:
#일단 1개라고 생각하고 돌리기
wine_index =df[df['name'].str.contains(wine_name)==True].index[0]
wine_index

0

In [11]:
def cosine_func(col_name,wine_index):
    
    #정규 표현식으로 영문만 남음
    only_english  = [ re.sub('[^a-zA-Z]', ' ',sentence).lower() for sentence in df[col_name]]
    #토큰만들기
    col_tokenized = [ nltk.word_tokenize(item) for item in only_english ] 
    #불용어 빼기
    no_stopwords = [ i for i in col_tokenized if i not in stopwords]
    #다시 합쳐주기
    final_review = [ ' '.join(item) for item in no_stopwords ]
    
    #Tfidf
    tfidf_vect = TfidfVectorizer()
    feature_vect = tfidf_vect.fit_transform(final_review)
    
    #코사인 유사도 분석 => 사용자가 입력한 와인과 전체 
    similarity_simple_pair = cosine_similarity(feature_vect[wine_index], feature_vect)
    result_list = similarity_simple_pair.tolist()[0]
    df[f'{col_name}_result'] = result_list
    
    return df

In [12]:
# 데이터프레임에 유사도 추가
my_col = ['foods','re']
for target_col in my_col:
    cosine_func(target_col,wine_index)

In [200]:
df.head()

,name,price,score,winery,grapes,country,region,alcohol,foods,wine_id,re,rev_alcohol,foods_result,re_result
0,Rim Rock Vineyard Syrah 2017,52.99,0.33,Piedrasassi,Shiraz/Syrah,United States,Arroyo Grande Valley,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",19603399,['Dry tingly cool bottle pairs with seafood ho...,0.70,1.000000,1.000000
1,Tradition Châteauneuf-du-Pape 2015,54.99,0.22,Domaine Giraud,Grenache,France,Châteauneuf-du-Pape,0.30,"['Lamb', 'Pork', 'Poultry']",18191466,"['Black cherry, oak. Leather, earthy. Great b...",0.70,0.334653,0.203839
2,Estate Cabernet Sauvignon 2013,73.95,0.33,Brandlin,Cabernet Sauvignon,United States,Mount Veeder,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Matu...",13886692,['Wine walk favorite! This estate cab is fant...,0.70,0.561375,0.167787
3,Imperio LXXIV Primitivo di Manduria 2018,19.99,0.22,Feudo Croce,Primitivo,Italy,Primitivo di Manduria,0.32,"['Beef', 'Pasta', 'Lamb', 'Poultry']",25311528,"['Plum, fig, pepper, cherry. ', 'Lots of dark ...",0.68,0.437166,0.196798
4,Corte di Cama Sforzato di Valtellina 2016,58.50,0.33,Mamete Prevostini,Nebbiolo,Italy,Sforzato della Valtellina,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",25588649,['A bit of a disappointment honestly. Have rea...,0.70,1.000000,0.141404


In [115]:
# result_arr = []
# # 유사도 열만 가져오기
# for column in df.columns.values :
#     if 'result' in column:
#         result_arr.append(column)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23394 entries, 0 to 23781
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          23394 non-null  object 
 1   price         23394 non-null  float64
 2   score         23394 non-null  float64
 3   winery        23394 non-null  object 
 4   grapes        23394 non-null  object 
 5   country       23394 non-null  object 
 6   region        23394 non-null  object 
 7   alcohol       23394 non-null  float64
 8   foods         23394 non-null  object 
 9   wine_id       23394 non-null  int64  
 10  re            23394 non-null  object 
 11  foods_result  23394 non-null  float64
 12  re_result     23394 non-null  float64
dtypes: float64(5), int64(1), object(7)
memory usage: 2.5+ MB


In [201]:
result_arr = []
for column in df.columns.values:
    if (df[column].dtype == 'float64') |  (df[column].dtype == 'int64'):
        if(column != 'wine_id'):
            result_arr.append(column)

In [202]:
result_arr

['price', 'score', 'alcohol', 'rev_alcohol', 'foods_result', 're_result']

In [203]:
df_result_weight = df[result_arr].copy()

In [204]:
df_result_weight.head()

,price,score,alcohol,rev_alcohol,foods_result,re_result
0,52.99,0.33,0.30,0.70,1.000000,1.000000
1,54.99,0.22,0.30,0.70,0.334653,0.203839
2,73.95,0.33,0.30,0.70,0.561375,0.167787
3,19.99,0.22,0.32,0.68,0.437166,0.196798
4,58.50,0.33,0.30,0.70,1.000000,0.141404


In [224]:
price_w = float(input("가중치를 입력해주세요.(합 1) \n가격 : "))
score_w = float(input("평점 : "))
alcohol_w = float(input("도수 : "))
food_w = float(input("어울리는 음식: "))
review_w = float(input("리뷰: "))

가중치를 입력해주세요.(합 1) 
가격 : 0
평점 : 0.1
도수 : 0.2
어울리는 음식: 0.3
리뷰: 0.4


In [225]:
# 가중치 추가

df['weight'] = df_result_weight.iloc[:,0]*price_w
df['weight']+=df_result_weight.iloc[:,1]*score_w
df['weight']+=df_result_weight.iloc[:,2]*alcohol_w
df['weight']+=df_result_weight.iloc[:,3]*food_w
df['weight']+=df_result_weight.iloc[:,4]*review_w 

In [226]:
df['weight']

0        0.703000
1        0.425861
2        0.527550
3        0.464866
4        0.703000
           ...   
23777    0.293000
23778    0.292000
23779    0.336513
23780    0.326000
23781    0.304000
Name: weight, Length: 23394, dtype: float64

In [227]:
df.head()

,name,price,score,winery,grapes,country,region,alcohol,foods,wine_id,re,rev_alcohol,foods_result,re_result,weight
0,Rim Rock Vineyard Syrah 2017,52.99,0.33,Piedrasassi,Shiraz/Syrah,United States,Arroyo Grande Valley,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",19603399,['Dry tingly cool bottle pairs with seafood ho...,0.70,1.000000,1.000000,0.703000
1,Tradition Châteauneuf-du-Pape 2015,54.99,0.22,Domaine Giraud,Grenache,France,Châteauneuf-du-Pape,0.30,"['Lamb', 'Pork', 'Poultry']",18191466,"['Black cherry, oak. Leather, earthy. Great b...",0.70,0.334653,0.203839,0.425861
2,Estate Cabernet Sauvignon 2013,73.95,0.33,Brandlin,Cabernet Sauvignon,United States,Mount Veeder,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Matu...",13886692,['Wine walk favorite! This estate cab is fant...,0.70,0.561375,0.167787,0.527550
3,Imperio LXXIV Primitivo di Manduria 2018,19.99,0.22,Feudo Croce,Primitivo,Italy,Primitivo di Manduria,0.32,"['Beef', 'Pasta', 'Lamb', 'Poultry']",25311528,"['Plum, fig, pepper, cherry. ', 'Lots of dark ...",0.68,0.437166,0.196798,0.464866
4,Corte di Cama Sforzato di Valtellina 2016,58.50,0.33,Mamete Prevostini,Nebbiolo,Italy,Sforzato della Valtellina,0.30,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",25588649,['A bit of a disappointment honestly. Have rea...,0.70,1.000000,0.141404,0.703000


In [228]:
df_sorted_by_values = df.sort_values(by='weight' ,ascending=False)

In [229]:
df_sorted_by_values[:5]

,name,price,score,winery,grapes,country,region,alcohol,foods,wine_id,re,rev_alcohol,foods_result,re_result,weight
14285,Au Paradis Cabernet Sauvignon 2016,224.99,0.78,Peter Michael,Cabernet Sauvignon,United States,Oakville,0.0,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",26260892,"['This is a smorgasbord of black berries, drie...",1.0,1.0,0.147203,0.778
12723,Château Margaux (Premier Grand Cru Classé) 1990,784.99,0.78,Château Margaux,Cabernet Sauvignon,France,Margaux,0.0,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",10073698,['Any description I could attempt to provide w...,1.0,1.0,0.125270,0.778
11396,Pauillac (Premier Grand Cru Classé) 1982,659.99,0.78,Château Mouton Rothschild,Cabernet Sauvignon,France,Pauillac,0.0,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",10097269,"['Imposing, powerful, evocative nose, promisin...",1.0,1.0,0.136551,0.778
17150,Estate Red 2016,169.99,0.78,Arkenstone,Cabernet Sauvignon,United States,Howell Mountain,0.0,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",21261266,"['Brombeeren, Kirschen. Etwas frischer. Tabak,...",1.0,1.0,0.007179,0.778
15693,Métisse Jumping Goat Vineyard 2016,184.99,0.78,Melka,Cabernet Sauvignon,United States,Napa Valley,0.0,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",25311688,"['Very nice blend - could use more cab', 'Yes!...",1.0,1.0,0.202638,0.778


In [236]:
df['wine_id'].value_counts()

18071498    2
17375715    2
14284518    2
726661      2
17198805    2
           ..
14738153    1
23849704    1
26472505    1
20726364    1
21366783    1
Name: wine_id, Length: 23267, dtype: int64

In [238]:
df[df['wine_id']==18071498]

,name,price,score,winery,grapes,country,region,alcohol,foods,wine_id,re,rev_alcohol,foods_result,re_result,weight
7992,Series M 2015,100.0,0.44,Vilafonté,Merlot,South Africa,Paarl,0.31,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",18071498,"['Very, very near to perfect. SA’s best winery...",0.69,1.0,0.088231,0.713
17442,Series M 2015,100.0,0.44,Vilafonté,Merlot,South Africa,Paarl,0.31,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",18071498,"['Very, very near to perfect. SA’s best winery...",0.69,1.0,0.088231,0.713
